# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [35]:
from selenium import webdriver #criar o navegador
from selenium.webdriver.common.keys import Keys#permite escrever no navegador
from selenium.webdriver.common.by import By#permite você selecionar itens no navegador
#abrir o navegador
navegador = webdriver.Chrome()

#passo 1: Pegar a acotação do dólar

#entrar no google
navegador.get("https://google.com.br")

#pesquisar no google a "cotação do dolar"
navegador.find_element("xpath", '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação dolar")
navegador.find_element("xpath", '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#pegar a cotação que tá no google
cotacao_dolar = navegador.find_element("xpath", '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

#passo 2: Pegar a cotação do euro
navegador.get("https://google.com.br")
navegador.find_element("xpath", '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação euro")
navegador.find_element("xpath", '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element("xpath", '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

#passo 3: Pegar a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element("xpath", '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",",".")

navegador.quit()

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [36]:
#passo 4: Atualizar a base de preços (atualizando o preço de compra e o de venda)
import pandas as pd
tabela=pd.read_excel("Produtos.xlsx")
display(data_base)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.1993,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6.0000,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.1993,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.1993,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6.0000,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.1993,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350.0000,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [37]:
#atualizar a coluna de cotação
#eu quero editar a coluan cotaçao onde a moeda e = dolar
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

#atualizar a coluna de preço de compra = preço original * cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

#atualizar a coluna de venda = preço de compra * margen
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.218413,5218.360816,1.40,7305.705142
1,Carro Renault,4500.00,Euro,5.113000,23008.500000,2.00,46017.000000
2,Notebook Dell,899.99,Dólar,5.218413,4696.519516,1.70,7984.083177
3,IPhone,799.00,Dólar,5.218413,4169.511987,1.70,7088.170378
4,Carro Fiat,3000.00,Euro,5.113000,15339.000000,1.90,29144.100000
5,Celular Xiaomi,480.48,Dólar,5.218413,2507.343078,2.00,5014.686156
6,Joia 20g,20.00,Ouro,288.910000,5778.200000,1.15,6644.930000


### Agora vamos exportar a nova base de preços atualizada

In [38]:
#passo 5: Exportar a base de preços atualizados
tabela.to_excel("Produtos Novo.xlsx", index=False)